## CulturaBot - création d'un chatbot sur la culture générale basé sur RAG destiné pour les ados/jeunes adultes

Le CulturaBot sera un chatbot qui utilisera les connaissances provenant du livre "La Culture générale pour les nuls" en version PDF pour répondre aux questions de culture générale de manière ludique et sans prétention. Ce projet se basera sur un modèle LLM extistant qui sera "augmenté" ou enrichi des connaissances dudit livre.

ETAPE 2 : Création des embeddings 

Le présent notebook a pour objectif de transformer le texte extrait du livre en vecteurs numériques porteur de valeur sématique. Vu le volume restreint du livre, nous n'utiliseront pas de base de données véctorielle externe.

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os

In [3]:
# 1. Charger le texte nettoyé
with open("../data/texte_nettoye.txt", "r", encoding="utf-8") as f:
    texte = f.read()

In [4]:
# 2. Découper le texte en chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
chunks = text_splitter.split_text(texte)
print(f"✅ Texte découpé en {len(chunks)} chunks.")

✅ Texte découpé en 2027 chunks.


In [5]:
# 3. Générer les embeddings avec un modèle français
#    --> Désactive la conversion automatique avec `encode_kwargs`
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)
print("🔍 Génération des embeddings en cours...")

C:\Users\aheij\AppData\Local\Temp\ipykernel_19340\615097066.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\aheij\Desktop\Jedha\Data Full Stack\Projects for Submission\06 Final Project bloc 6\projet-culture-g\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔍 Génération des embeddings en cours...


In [7]:
# 4. Créer et sauvegarder l'index FAISS
db = FAISS.from_texts(chunks, embeddings)
db.save_local("../data/faiss_index")
print("🎉 Embeddings générés et sauvegardés dans data/faiss_index/")

🎉 Embeddings générés et sauvegardés dans data/faiss_index/


In [8]:
# 5. Afficher un exemple de chunk
if chunks:
    print("\n--- Exemple de chunk ---")
    print(chunks[0][:200] + "...")




--- Exemple de chunk ---
Avec les Nuls, tout devient facile ! 2e édition La Culture générale cid23cid3Un panorama complet de tous les domaines du savoir cid23cid3Des milliers de faits, de dates et de noms à retenir sans peine...


In [10]:
# Afficher un exemple d'embedding
if db.index.ntotal > 0:
    print("\n--- Exemple d'embedding ---")
    print(db.index.reconstruct(0)[:10], "...")


--- Exemple d'embedding ---
[-0.03595955  0.04874558  0.01961184  0.01233185  0.00050273  0.03276659
 -0.04234035  0.03906041 -0.03726172  0.08657563] ...
